In [ ]:
import os, sys
import pandas as pd
import numpy as np
import cv2
import shutil
import struct

In [ ]:
HOME = os.path.expanduser("~")
PATH = os.path.join(HOME, 'programming/pipeline_utility')
sys.path.append(PATH)
from utilities.file_location import FileLocationManager

In [ ]:
DATA_PATH = '/net/birdstore/Active_Atlas_Data/data_root'
csvfile = os.path.join(DATA_PATH, 'atlas_data', 'DK52.all.csv')
DK52_df = pd.read_csv(csvfile, dtype={'Layer':'str', 'X':np.uint16, 'Y':np.uint16,'Section':np.uint16})
DK52_df = DK52_df.loc[DK52_df['Layer'] == 'PM nucleus']
animal = 'DK52'
outpath = os.path.join(DATA_PATH, 'atlas_data', 'DK52.PM.Nucleus.csv')
DK52_df.to_csv(outpath, index=False)
fileLocationManager = FileLocationManager(animal)
OUTPUT_DIR = os.path.join(fileLocationManager.neuroglancer_data, 'points')
if os.path.exists(OUTPUT_DIR):
    shutil.rmtree(OUTPUT_DIR)
os.makedirs(OUTPUT_DIR, exist_ok=True)

In [ ]:
coordinates = []
for index, row in DK52_df.iterrows():
    x = row['X']
    y = row['Y']
    z = row['Section']
    coordinates.append((x,y,z))

1. This takes the coordinates and packs them into a binary file, 
2. see https://github.com/google/neuroglancer/issues/227
3. Create a dir on birdstore called points
4. put the info file under points/info
5. create the binary file and put in points/spatial0/0_0_0

In [ ]:
spatial_dir = os.path.join(OUTPUT_DIR, 'spatial0')
os.makedirs(spatial_dir)

with open(os.path.join(spatial_dir, '0_0_0'), 'wb') as outfile:
    total_count=len(coordinates) # coordinates is a list of tuples (x,y,z) 
    buf = struct.pack('<Q',total_count)
    for (x,y,z) in coordinates:
        pt_buf = struct.pack('<3f',x,y,z)
        buf+=pt_buf
    # write the ids at the end of the buffer as increasing integers 
    id_buf = struct.pack('<%sQ' % len(coordinates), *range(len(coordinates)))
    buf+=id_buf
    outfile.write(buf)